<a href="https://colab.research.google.com/github/yujungha/Data_Analysis/blob/main/16_1%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
def dateformat(y=0,m=0,d=0,h=0,mm=0,s=0):
    return "%d-%02d-%02d %02d:%02d:%02d"%(y,m,d,h,mm,s)

In [4]:
# txt인 경우 (iphone)
f = open('Talk_2021.4.29 21_54-1.txt', mode='r', encoding='utf-8')

file_name = f.readline()
save_date_re = re.compile('저장한 날짜 : ([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2})')
m = save_date_re.match(f.readline())
save_date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))

f.readline() # 공백 제거
f.readline() # 공백 제거
f.readline() # 공백 제거

history = []
histline = ''
date = None

date_re = re.compile('([0-9]{4})년 ([0-9]{1,2})월 ([0-9]{1,2})일 ((월|화|수|목|금|토|일)요일)')
invite_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}): ((.+)님이 .+ 초대했습니다.)')
exit_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}): ((.+)님이 나갔습니다.)')
chat_re = re.compile('([0-9]{4})\. ([0-9]{1,2})\. ([0-9]{1,2})\. ([0-9]{2}):([0-9]{2}), (.+) : (.+)')

chat_temp = ''

i=0
while True:
    line = f.readline()
    if not line: 
        break
    # print(line, end='')

    # chat
    m = chat_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
        user = m[6]
        content = m[7]
        history.append([date, user, content])
        continue
    # date
    m = date_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]))
        # print('aaaaaaaaaaaa' ,history[-1][2])
        continue
    # invite
    m = invite_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
        user = m[7]
        content = m[6]
        history.append([date, user, content])
        continue
    # exit
    m = exit_re.match(line)
    if m:
        date = dateformat(int(m[1]), int(m[2]), int(m[3]), int(m[4]), int(m[5]))
        user = m[7]
        content = m[6]
        history.append([date, user, content])
        continue
    # \n chat
    history[-1][2] += line
df = pd.DataFrame(history, columns=['Date', 'User', 'Message'])

In [ ]:
df

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     4717 non-null   object
 1   User     4717 non-null   object
 2   Message  4717 non-null   object
dtypes: object(3)
memory usage: 110.7+ KB


In [ ]:
a = df['Message'].values
b = []
for i in range(len(a) - 1):
  b.append([a[i], a[i + 1]])
b

In [8]:
len(a)

4717

In [ ]:
a = np.array(df[['User', 'Message']].values)
# a = [['19 김은하', '19 김은하님이 누리고라스님과 유정하님을 초대했습니다.'],
#     ['19 김은하', '애들아'],
#     ['19 김은하', '조이름생각해바']]
i = 0
while i != len(a)-1:
  if a[i][0] == a[i+1][0]:
    a[i][1] += ' '+a[i+1][1]
    a = np.delete(a, i+1, 0)
  else:
    i += 1

a = a.T[1]

b = []
for i in range(len(a) - 1):
  b.append([a[i], a[i + 1]])
b

In [ ]:
import random

while True:
  Q = input("질문 : ")
  l1 = []
  l2 = []
  l3 = []

  for i in range(len(b)):
    if Q == b[i][0]:
      # print('[완벽일치]', b[i][0])
      l1.append(b[i][1])
    elif b[i][0].find(Q) != -1:
      # print('[포함]',b[i][0])
      l2.append(b[i][1])
    # [포함-2] 채팅
    else:
      n = 0
      for q in Q.split(' '):
        if b[i][0].find(q) != -1:
          n += 1
      if n > 0:
        # print('[포함-%d]'%n,b[i][0])
        l3.append(b[i][1])

  if len(l1) != 0:
    print('답변 :',random.choice(l1))
  elif len(l2) != 0:
    print('답변 :',random.choice(l2))
  elif len(l3) != 0:
    print('답변 :',random.choice(l3))
  else:
    print('답변 : 맞아맞아')
  print('\n=================================================\n')